_Última modificación: 27 de noviembre de 2024_
# Práctica 4: Identificación de palabras, frases, y documentos similares
**Hernández Jiménez Erick Yael**: 2023630748.

Escuela Superior de Cómputo.

Ingeniería en inteligencia Artificial. 5BV1.

Tecnologías para el Procesamiento de Lenguaje Natural.

## Resumen
En este cuaderno se usará el procesamiento de cuerpos para identificar palabras, frases y documentos similares de un cuerpo de 5 documentos tras normalizarlos. A continuación se enlistan las bibliotecas utilizadas.

In [1]:
import math             # Para operaciones matemáticas complejas
import nltk             # Biblioteca con las herramientas utilizadas para la manipulación y procesamiento de los documentos
import re               # Para eliminar saltos de línea
import torch
import numpy as np      # Biblioteca auxiliar

from nltk import pos_tag                                    # Para etiquetar tokens por función gramaticar
from nltk.corpus import stopwords, wordnet                  # Para stop-words
from nltk.stem import WordNetLemmatizer                     # Para lematización
from nltk.tokenize import word_tokenize, sent_tokenize      # Submódulo para tokenizar (nltk)
from numpy.typing import NDArray                            # Para documentar las variables de Numpy
from rake_nltk import Rake                                  # Algoritmo Rake modificado
from sumy.nlp.tokenizers import Tokenizer                   # Para tokenizar (sumy)
from sumy.parsers.plaintext import PlaintextParser          # Para adaptar las cadenas de texto
from sumy.summarizers.text_rank import TextRankSummarizer   # Para TextRank
from scipy.spatial.distance import cosine                   # Para calcular la distancia cosenoidal
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mean                                 # Para calcular automáticamente la media
from transformers import BertTokenizer, BertModel           # Para los modelos de BERT
# from transformers import pipeline                           # Para usar BERT
# Archivos auxiliares para el funcionamiento de nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')                                      
nltk.download('stopwords')
nltk.download('wordnet')

/home/hjey/Documentos/TPLN/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/hjey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/hjey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hjey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hjey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Generación de cuerpo de documentos
Extrayendo el primer capítulo o introducción de 5 libros de Julio Verne, desde el enlace [“https://www.gutenberg.org/ebooks/84”](“https://www.gutenberg.org/ebooks/84”), los textos se guardaron manualmente como archivos de texto en la carpeta docs como:
- [A_JOURNEY_TO_THE_CENTRE_OF_THE_EARTH_CHAPTER_1](./docs/A_JOURNEY_TO_THE_CENTRE_OF_THE_EARTH_CHAPTER_1.txt)
- [THE_MYSTERIOUS_ISLAND_CHAPTER_1](./docs/THE_MYSTERIOUS_ISLAND_CHAPTER_1.txt)
- [TWENTY_THOUSAND_LEAGUES_UNDER_THE_SEA_CHAPTER_1](./docs/TWENTY_THOUSAND_LEAGUES_UNDER_THE_SEA_CHAPTER_1.txt)
- [OFF_ON_A_COMET_CHAPTER_1](./docs/OFF_ON_A_COMET_CHAPTER_1.txt)
- [ALL_AROUND_THE_MOON_CHAPTER_1](./docs/ALL_AROUND_THE_MOON_CHAPTER_1.txt)

In [2]:
paths: list[str] = [
    "docs/A_JOURNEY_TO_THE_CENTRE_OF_THE_EARTH_CHAPTER_1.txt",
    "docs/THE_MYSTERIOUS_ISLAND_CHAPTER_1.txt",
    "docs/TWENTY_THOUSAND_LEAGUES_UNDER_THE_SEA_CHAPTER_1.txt",
    "docs/OFF_ON_A_COMET_CHAPTER_1.txt",
    "docs/ALL_AROUND_THE_MOON_CHAPTER_1.txt"
]

libros: list[str] = []
for path in paths:
    with open(path, 'r', encoding="utf-8") as file:
        libros.append(file.read())

for libro in libros:
    print(libro[:50])

MY UNCLE MAKES A GREAT DISCOVERY

Looking back to 
 “Are we rising again?” “No. On the contrary.” “Ar
A SHIFTING REEF

The year 1866 was signalised by a
 CHAPTER I. A CHALLENGE

“Nothing, sir, can induce
FROM 10 P.M. TO 10 46' 40''.

The moment that the 


## Normalización de textos general
El flujo que se usará, y su justificación se explicará a continuación:
1. Tokenización por enunciados: Se seguirá con la indicación del desarrollo y se aplicará inicialmente la tokenización para continuar con...
2. Tokenización por palabras: Siguiendo las indicaciones, tokenizar por palabras por enunciado permite mantener el orden y tokens por enunciado, siendo equivalente sustancialmente a los textos originles.
3. Etiqueta por posición y función gramatical: Se siguen indicaciones y aplicarlo tras el paso anterior es mandatorio para el correcto etiquetado.
4. Conversión a minúsculas: para evitar redundancias en el contenido significativo del cuerpo.
5. Filtrar caracteres alfabéticos: para únicamente conservarlos y, así, reducir el ruido en el análisis ocasionado por caracteres que no son de interés ni aportan significados relevantes para el mismo análisis.

Se importarán los constructores y funciones necesarios para este proceso.

In [3]:
lemmatizer: WordNetLemmatizer = WordNetLemmatizer() # Constructor para lematizar
stop_words = set(stopwords.words('english'))        # Set con las stop-words del inglés (idioma en el que están los documentos)

In [4]:
def get_wordnet_pos(treebank_tag):
    """Convierte etiquetas POS de Penn Treebank a WordNet."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Predeterminado: sustantivo

In [5]:
# Almacenamos los libros en una misma lista, este será el corpus
corpus_per_book: list[list[str]] = []

# Por cada libro `book` `i` que se encuentran en `libros`...
for i, book in enumerate(libros):
    # tokenizamos el libro por enunciados
    sentences: list[str] = sent_tokenize(book)
    # almacenamos las palabras etiquetadas en una lista...
    tagged_words: list[list[tuple[str, str]]] = [
        # con el resultado del etiquetado...
        pos_tag(
            # con las palabras tokenizadas...
            word_tokenize(
                # de cada enunciado `sentence`...
                sentence)) 
        # para cada `sentence` en los enunciados del libro `sentences`
        for sentence in sentences]

    # almacenamos las palabras normalizadas en una lista...
    normalized_words: list[str] = []
    # por cada enunciado `sentence` en las palabras etiquetadas `tagged_words`...
    for sentence in tagged_words:
        # almacenamos en una lista...
        normalized: list[tuple[str, str]] = [
            # el resutado de lematizar cada palabra `word` en minúsculas en cada enunciado `sentence`...
            (lemmatizer.lemmatize(word.lower(), get_wordnet_pos(tag)), get_wordnet_pos(tag)) for word, tag in sentence
            # si cumple con que es un caractér alfabético y si no se encuentra en el set de `stop_words`
            if word.isalpha() and word.lower() not in stop_words
        ]
        # extendemos el contenido a las palabras normalizadas del libro
        normalized_words.extend(normalized)
    # agregamos los tokens del libro al corpus
    corpus_per_book.append(normalized_words)

# Imprimimos los resultados
for book in corpus_per_book:
    print(book)

[('uncle', 'n'), ('make', 'n'), ('great', 'n'), ('discovery', 'n'), ('looking', 'n'), ('back', 'r'), ('occur', 'v'), ('since', 'n'), ('eventful', 'a'), ('day', 'n'), ('scarcely', 'r'), ('able', 'a'), ('believe', 'v'), ('reality', 'n'), ('adventure', 'n'), ('truly', 'r'), ('wonderful', 'a'), ('even', 'r'), ('bewilder', 'v'), ('think', 'v'), ('uncle', 'n'), ('german', 'a'), ('marry', 'v'), ('mother', 'n'), ('sister', 'n'), ('englishwoman', 'n'), ('much', 'r'), ('attach', 'v'), ('fatherless', 'n'), ('nephew', 'r'), ('invite', 'v'), ('study', 'v'), ('home', 'n'), ('fatherland', 'n'), ('home', 'n'), ('large', 'a'), ('town', 'n'), ('uncle', 'n'), ('professor', 'n'), ('philosophy', 'n'), ('chemistry', 'n'), ('geology', 'n'), ('mineralogy', 'n'), ('many', 'a'), ('ology', 'n'), ('one', 'n'), ('day', 'n'), ('pass', 'v'), ('hour', 'n'), ('uncle', 'n'), ('absent', 'n'), ('suddenly', 'r'), ('felt', 'v'), ('necessity', 'n'), ('renovate', 'v'), ('hungry', 'a'), ('rouse', 'v'), ('old', 'a'), ('french'

## Similitud de palabras con synsets
Tomando el ejemplo que se encuentra en la [documentación de nltk](https://www.nltk.org/howto/wordnet.html#similarity) como guía:
Ya que tenemos cada palabra etiqueta por su POS y en su lema más básico...:
1. Por cada libro crearemos un diccionario con los verbos y las frecuencias de cada uno.
2. Seleccionamos el verbo más común por ser el más frecuente
3. Buscamos los 5 verbos más comunes en el libro correspondiente con "wup similarity"
4. Buscamos los 5 verbos más comunes en el libro correspondiente con "path similarity"

Así mismo...:
Por cada libro crearemos un diccionario con los sustantivos y las frecuencias de cada uno.
2. Seleccionamos el sustantivo más común por ser el más frecuente
3. Buscamos los 5 sustantivos más comunes en el libro correspondiente con "wup similarity"
4. Buscamos los 5 sustantivos más comunes en el libro correspondiente con "path similarity"
 

In [6]:
# VERB SIMILARITY
# Almacenamos los verbos por frecuencias en una lista que almacene el diccionario correspondiente a los verbos de cada libro
most_freq_verbs: list[dict[str, int]] = []

# Por cada libro en el corpus...
for book in corpus_per_book:
    # almacenamos el ranking de verbos en un diccionario con el verbo como llave `key` y su frecuencia como valor `value`
    verb_ranking: dict[str, int] = {}
    # por cada verbo `verb` y etiqueta `tag` en la lista con...
    for verb, tag in [
                    # los elementos `element` para cada elemento en el libro `book`...
                    element for element in book 
                    # si la etiqueta de la palabra en el elemento es 'v'
                    if element[1] == 'v']:
        # si existe el verbo en las llaves `keys` del diccionario...
        if verb in verb_ranking.keys():
            # se le suma uno al conteo de los verbos
            verb_ranking[verb] += 1
        # de lo contrario...
        else:
            # se agrega el verbo y se inicia la cuenta
            verb_ranking[verb] = 1
    # agregamos el diccionario ordenado descendentemente por las frecuencias de los verbos
    most_freq_verbs.append(dict(sorted(verb_ranking.items(), key=lambda x: x[1], reverse=True)))

# Imprimimos los resultados
for book in most_freq_verbs:
    print(book)

{'say': 7, 'know': 7, 'make': 7, 'come': 5, 'find': 5, 'mean': 4, 'cry': 4, 'declare': 4, 'tell': 3, 'learn': 3, 'give': 3, 'take': 3, 'grow': 3, 'think': 2, 'study': 2, 'open': 2, 'obey': 2, 'wait': 2, 'consume': 2, 'pore': 2, 'stammer': 2, 'resemble': 2, 'hop': 2, 'see': 2, 'go': 2, 'relate': 2, 'love': 2, 'appear': 2, 'write': 2, 'fell': 2, 'occur': 1, 'believe': 1, 'bewilder': 1, 'marry': 1, 'attach': 1, 'invite': 1, 'pass': 1, 'felt': 1, 'renovate': 1, 'rouse': 1, 'rush': 1, 'bear': 1, 'resound': 1, 'shout': 1, 'attend': 1, 'hasten': 1, 'reach': 1, 'jump': 1, 'stamp': 1, 'constitute': 1, 'soup': 1, 'adjourn': 1, 'present': 1, 'lay': 1, 'diffuse': 1, 'digest': 1, 'keep': 1, 'acquire': 1, 'regard': 1, 'object': 1, 'display': 1, 'explain': 1, 'sun': 1, 'star': 1, 'comprehend': 1, 'replace': 1, 'use': 1, 'improve': 1, 'swallow': 1, 'add': 1, 'bind': 1, 'prefer': 1, 'gain': 1, 'break': 1, 'classify': 1, 'correspond': 1, 'wish': 1, 'confer': 1, 'compare': 1, 'attract': 1, 'step': 1, 'cl

In [7]:
# Por cada ranking en la lista `most_freq_verbs`...
for ranking in most_freq_verbs:
    # obtenemos el verbo más frecuente
    top_word = max(ranking, key=ranking.get)
    # obtenemos su synset
    top_synsets = wordnet.synsets(top_word)
    # si no hay synsets...
    if not top_synsets:
        # se continúa con la siguiente palabra
        print(f"No hay sinsets para el verbo '{top_word}'")
        continue
    # escogemos la primera coincidencia
    top_synset = top_synsets[0]

    # Calcular similitudes Wu-Palmer con otros verbos en el ranking en una lista ordenada...
    results_wup = sorted([
        # con tuplas con el verbo `verb` y el puntaje dado por evaluar la similitud de la palabra más frecuente `top_synset`...
        (verb, top_synset.wup_similarity(
            # con el primer synset del verbo `verb` de la iteración
            wordnet.synsets(verb)[0]) 
                # si hay synsets para el verbo, de lo contrario, se retorna 0
                if wordnet.synsets(verb) else 0)
        # para cada verbo `verb` en las llaves del diccionario `ranking`
        for verb in ranking.keys()
    # y usamos el porcentaje de coincidencia para ordenar descendentemente...
    ], key=lambda item: item[1], reverse=True
        # y rescatamos los elementos 1 al 6 para no tomar en cuenta la palabra propia
        )[1:6]

    # Calcular similitudes Path con otros verbos en el ranking en una lista ordenada...
    results_path = sorted([
        # con tuplas con el verbo `verb` y el puntaje dado por evaluar la similitud de la palabra más frecuente `top_synset`...
        (verb, top_synset.path_similarity(
            # con el primer synset del verbo `verb` de la iteración
            wordnet.synsets(verb)[0]) 
                # si hay synsets para el verbo, de lo contrario, se retorna 0
                if wordnet.synsets(verb) else 0)
        # para cada verbo `verb` en las llaves del diccionario `ranking`
        for verb in ranking.keys()
    # y usamos el porcentaje de coincidencia para ordenar descendentemente...
    ], key=lambda item: item[1], reverse=True
        # y rescatamos los elementos 1 al 6 para no tomar en cuenta la palabra propia
        )[1:6]

    # Imprimimos los resultados
    print(f"Las palabras más similares a {top_word} ({top_synset.name()}) con el método...\n {" wu palmer ".center(30, '-')}")
    for result in results_wup:
        print(result)
    print(" path similarity ".center(30, '-'))
    for result in results_path:
        print(result)
    print('*'*30, "\n")


Las palabras más similares a say (say.n.01) con el método...
 --------- wu palmer ----------
('love', 0.5333333333333333)
('wish', 0.5333333333333333)
('delight', 0.5333333333333333)
('surprise', 0.5333333333333333)
('want', 0.5)
------ path similarity -------
('love', 0.125)
('present', 0.125)
('wish', 0.125)
('delight', 0.125)
('surprise', 0.125)
****************************** 

Las palabras más similares a throw (throw.n.01) con el método...
 --------- wu palmer ----------
('drive', 0.8571428571428571)
('try', 0.7142857142857143)
('escape', 0.7142857142857143)
('support', 0.7142857142857143)
('pass', 0.6666666666666666)
------ path similarity -------
('drive', 0.3333333333333333)
('try', 0.2)
('escape', 0.2)
('support', 0.2)
('pass', 0.16666666666666666)
****************************** 

Las palabras más similares a go (go.n.01) con el método...
 --------- wu palmer ----------
('grave', 0.375)
('hundred', 0.35294117647058826)
('thousand', 0.35294117647058826)
('signal', 0.30769230769

In [8]:
# NOUN SIMILARITY
# Almacenamos los sustantivos por frecuencias en una lista que almacene el diccionario correspondiente a los sustantivos de cada libro
most_freq_nouns: list[dict[str, int]] = []

# Por cada libro en el corpus...
for book in corpus_per_book:
    # almacenamos el ranking de sustantivos en un diccionario con el sustantivo como llave `key` y su frecuencia como valor `value`
    noun_ranking: dict[str, int] = {}
    # por cada sustantivo `noun` y etiqueta `tag` en la lista con...
    for noun, tag in [
                    # los elementos `element` para cada elemento en el libro `book`...
                    element for element in book 
                    # si la etiqueta de la palabra en el elemento es 'n'
                    if element[1] == 'n']:
        # si existe el sustantivo en las llaves `keys` del diccionario...
        if noun in noun_ranking.keys():
            # se le suma uno al conteo de los sustantivos
            noun_ranking[noun] += 1
        # de lo contrario...
        else:
            # se agrega el sustantivo y se inicia la cuenta
            noun_ranking[noun] = 1
    # agregamos el diccionario ordenado descendentemente por las frecuencias de los sustantivos
    most_freq_nouns.append(dict(sorted(noun_ranking.items(), key=lambda x: x[1], reverse=True)))

# Imprimimos los resultados
for book in most_freq_nouns:
    print(book)

{'uncle': 33, 'professor': 8, 'one': 8, 'hardwigg': 6, 'time': 6, 'house': 6, 'man': 5, 'dinner': 5, 'mineralogy': 4, 'cook': 4, 'upon': 4, 'like': 4, 'event': 4, 'day': 3, 'adventure': 3, 'geology': 3, 'three': 3, 'truth': 3, 'question': 3, 'science': 3, 'would': 3, 'glass': 3, 'interest': 3, 'u': 3, 'letter': 3, 'subject': 3, 'nothing': 3, 'every': 3, 'icelandic': 3, 'language': 3, 'home': 2, 'town': 2, 'street': 2, 'door': 2, 'foot': 2, 'could': 2, 'room': 2, 'tone': 2, 'moment': 2, 'omelette': 2, 'value': 2, 'benefit': 2, 'order': 2, 'knowledge': 2, 'word': 2, 'connection': 2, 'fond': 2, 'fact': 2, 'study': 2, 'specimen': 2, 'meal': 2, 'men': 2, 'nose': 2, 'article': 2, 'presence': 2, 'half': 2, 'notion': 2, 'part': 2, 'four': 2, 'leaf': 2, 'account': 2, 'hand': 2, 'book': 2, 'work': 2, 'iceland': 2, 'idiom': 2, 'parchment': 2, 'piece': 2, 'runic': 2, 'matter': 2, 'two': 2, 'thousand': 2, 'make': 1, 'great': 1, 'discovery': 1, 'looking': 1, 'since': 1, 'reality': 1, 'mother': 1, 's

In [9]:
# Por cada ranking en la lista `most_freq_nouns`...
for ranking in most_freq_nouns:
    # obtenemos el sustantivo más frecuente
    top_word = max(ranking, key=ranking.get)
    # obtenemos su synset
    top_synsets = wordnet.synsets(top_word)
    # si no hay synsets...
    if not top_synsets:
        # se continúa con la siguiente palabra
        print(f"No hay sinsets para el sustantivo '{top_word}'")
        continue
    # escogemos la primera coincidencia
    top_synset = top_synsets[0]

    # Calcular similitudes Wu-Palmer con otros sustantivos en el ranking en una lista ordenada...
    results_wup = sorted([
        # con tuplas con el sustantivo `noun` y el puntaje dado por evaluar la similitud de la palabra más frecuente `top_synset`...
        (noun, top_synset.wup_similarity(
            # con el primer synset del sustantivo `noun` de la iteración
            wordnet.synsets(noun)[0]) 
                # si hay synsets para el sustantivo, de lo contrario, se retorna 0
                if wordnet.synsets(noun) else 0)
        # para cada sustantivo `noun` en las llaves del diccionario `ranking`
        for noun in ranking.keys()
    # y usamos el porcentaje de coincidencia para ordenar descendentemente...
    ], key=lambda item: item[1], reverse=True
        # y rescatamos los elementos 1 al 6 para no tomar en cuenta la palabra propia
        )[1:6]

    # Calcular similitudes Path con otros verbos en el ranking en una lista ordenada...
    results_path = sorted([
        # con tuplas con el verbo `verb` y el puntaje dado por evaluar la similitud de la palabra más frecuente `top_synset`...
        (noun, top_synset.path_similarity(
            # con el primer synset del sustantivo `noun` de la iteración
            wordnet.synsets(noun)[0]) 
                # si hay synsets para el sustantivo, de lo contrario, se retorna 0
                if wordnet.synsets(noun) else 0)
        # para cada sustantivo `noun` en las llaves del diccionario `ranking`
        for noun in ranking.keys()
    # y usamos el porcentaje de coincidencia para ordenar descendentemente...
    ], key=lambda item: item[1], reverse=True
        # y rescatamos los elementos 1 al 6 para no tomar en cuenta la palabra propia
        )[1:6]

    # Imprimimos los resultados
    print(f"Las palabras más similares a {top_word} ({top_synset.name()}) con el método...\n {" wu palmer ".center(30, '-')}")
    for result in results_wup:
        print(result)
    print(" path similarity ".center(30, '-'))
    for result in results_path:
        print(result)
    print('*'*30, "\n")

Las palabras más similares a uncle (uncle.n.01) con el método...
 --------- wu palmer ----------
('person', 0.7058823529411765)
('relative', 0.6666666666666666)
('inhabitant', 0.6666666666666666)
('polyglot', 0.6666666666666666)
('man', 0.631578947368421)
------ path similarity -------
('relative', 0.3333333333333333)
('person', 0.25)
('inhabitant', 0.2)
('polyglot', 0.2)
('man', 0.16666666666666666)
****************************** 

Las palabras más similares a balloon (balloon.n.01) con el método...
 --------- wu palmer ----------
('ship', 0.782608695652174)
('car', 0.6666666666666666)
('ammunition', 0.6)
('machine', 0.6)
('basket', 0.6)
------ path similarity -------
('ship', 0.16666666666666666)
('car', 0.1111111111111111)
('surface', 0.1111111111111111)
('layer', 0.1111111111111111)
('plaything', 0.1111111111111111)
****************************** 

No hay sinsets para el sustantivo 'could'
Las palabras más similares a captain (captain.n.01) con el método...
 --------- wu palmer ---

## Similitud de documentos con synsets
En la práctica 3 se encontraron 3 métodos con buenos resultados: Text Rank, Rake y Frecuencia de Palabras Normalizada. Se usará Rake ya que solo se requiere unir los tokens de cada libro, simplificando el proceso.

In [10]:
alg_rake: Rake = Rake()

In [11]:
# Almacenamos las frases más significativas de cada libro en una lista:
top_sentences: list[list[str]] = []

# Por cada libro `book` en `libros`
for book in libros:
    # Eliminar números y normalizar espacios
    clean_book = re.sub(r'\d+', '', book)           # Eliminar números
    clean_book = clean_book.replace('\n', ' ')      # Reemplazar saltos de línea por espacios
    clean_book = clean_book.replace('!', '.')       # Elimina el caracter '!'
    clean_book = clean_book.replace('?', '.')       # Elimina el caracter '?'
    clean_book = clean_book.replace('”', '')        # Elimina el caracter '”'
    clean_book = clean_book.replace('“', '')        # Elimina el caracter '“'
    clean_book = clean_book.replace('—', '')        # Elimina el caracter '—'
    clean_book = re.sub(r'\s+', ' ', clean_book)    # Reemplazar múltiples espacios por uno solo

    # Tokenizar en oraciones
    sentences: list[str] = sent_tokenize(clean_book)
    # print("\n" + "Oraciones tokenizadas:".center(150, '-'))
    # for sentence in sentences:
    #     print(sentence)

    # Procesamos las oraciones unidas por un espacio ' ' con el algoritmo rake
    alg_rake.extract_keywords_from_text(" ".join(sentences))
    # Almacenar la oración principal para este libro
    top_sentences.append(alg_rake.get_ranked_phrases()[0])

# Imprimimos los resultados
for top in top_sentences:
    print(top)

classify six hundred different geological specimens
several thousand people crushed
immense kraken whose tentacles could entangle
two officers listened gravely enough
car soon came back empty


In [12]:

# Tokenizar las frases y obtener sus sinsets
def get_synsets(sentence):
    words = word_tokenize(sentence)
    synsets = []
    for word in words:
        # Obtener el primer sinset disponible para cada palabra
        synset = wordnet.synsets(word)
        if synset:
            synsets.append(synset[0])  # Seleccionar el primer sinset por simplicidad
    return synsets

# Calcular similitud promedio entre dos listas de sinsets
def calculate_path_similarity(synsets1, synsets2):
    similarities = []
    for s1 in synsets1:
        for s2 in synsets2:
            similarity = s1.path_similarity(s2)
            if similarity:  # Ignorar valores `None`
                similarities.append(similarity)
    return mean(similarities) if similarities else 0  # Retornar promedio o 0 si no hay similitudes

# Seleccionar el libro de referencia (el primero en este ejemplo)
reference_sentence = top_sentences[0]
reference_synsets = get_synsets(reference_sentence)

# Comparar con las demás frases
for i, sentence in enumerate(top_sentences[1:], start=1):
    # Obtenemos los synsets del enunciado
    comparison_synsets = get_synsets(sentence)
    # Obtenemos la similitud entre el primer libro y el enunciado del otro libro de la iteración
    similarity = calculate_path_similarity(reference_synsets, comparison_synsets)
    # Imprimimos los resultados
    print(f"Similitud entre '{reference_sentence}' y libro {i}({sentence}): {similarity:.2f}")


Similitud entre 'classify six hundred different geological specimens' y libro 1(several thousand people crushed): 0.17
Similitud entre 'classify six hundred different geological specimens' y libro 2(immense kraken whose tentacles could entangle): 0.12
Similitud entre 'classify six hundred different geological specimens' y libro 3(two officers listened gravely enough): 0.14
Similitud entre 'classify six hundred different geological specimens' y libro 4(car soon came back empty): 0.12


## Similitud de palabras con “embedding”
Para este método, es necesario descargar el modelo manualmente. Para este desarrollo particular, se descargará en el mismo entorno virtual con el que se desarrolla toda la práctica general, ingresando los comandos:
```bash
wget http://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip
```
>**[NOTA]** Los archivos se descargaron y descomprimieron por defecto en la dirección del entorno, y se movieron manualmente a la carpeta [`glove`](./glove) para facilitar su acceso en el desarrollo del código. Si se cambia la dirección del archivo `glove.6B.300d.txt`, será necesario reflejarlo en la línea comentada correspondiente o mover el archivo a la misma dirección en la que se encuentra el presente archivo.

Se escogerá el modelo "glove.6B.300d.txt" para mayor detalle en el análisis de similitud de los textos

In [13]:
# Definimos una función para cargar el modelo
def load_glove_model(glove_file):
    print("Cargando el modelo GloVe...")
    # Almacenamos los embeddings en un diccionario
    embeddings: dict = {}
    # Abrimos el archivo `glove_file` en modo lecura `r` como `f`
    with open(glove_file, 'r', encoding='utf-8') as f:
        # Para cada línea `line` en el archivo...
        for line in f:
            # partimos la línea en partes `parts`
            parts: list[str] = line.split()
            # tomamos la primera palabra y la almacenamos en `word`
            word = parts[0]
            # Generamos el vector correspondiente a la palabra `word` con los valores correspondientes en las partes siguientes
            vector = np.array(parts[1:], dtype=np.float32)
            # Agregamos la palabra y su vector a los embeddings
            embeddings[word] = vector
    # Finalizamos la carga regresando el diccionario con los embeddings
    print("Modelo GloVe cargado.")
    return embeddings

In [14]:
# Cargamos el modelo de interés
glove_model = load_glove_model(fr"./glove/glove.6B.300d.txt") # Modificar la dirección del archivo si es necesario

Cargando el modelo GloVe...
Modelo GloVe cargado.


In [15]:
# Definimos la función que retornará la diferencia de similitud
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

In [16]:
# Almacenamos las palabras similares en una lista
similar_words: list = []

# Para cada libro `book` `idx` al enumerar los elementos del diccionario con los verbos más frecuentes de cada libro `most_freq_verbs`
for idx, book in enumerate(most_freq_verbs):
    # Tomamos el verbo con más frecuencia
    most_frequent_verb = list(most_freq_verbs[idx].keys())[0]  # Verbo más frecuente
    # Si el verbo se encuentra en el modelo...
    if most_frequent_verb in glove_model:
        # obtenemos el vector
        verb_vector = glove_model[most_frequent_verb]
        # guardamos las similitudes de cada palabra en un diccionario
        word_similarities: dict = {}
        # por cada palabra `word` en el set de las palabras del libro `dx`...
        for word in set(item[0] for item in corpus_per_book[idx]): 
            # si la palabra se encuentra en el modelo...
            if word in glove_model:
                # obtenemos el vector de la palabra
                word_vector = glove_model[word]
                # se obtiene la similitud entre ambas palabras
                similarity = cosine_similarity(verb_vector, word_vector)
                # se guarda la similitud en el diccionario de similitudes
                word_similarities[word] = similarity
        
        # se obtienen las 5 palabras más similares
        top_similar = sorted(word_similarities.items(), key=lambda x: x[1], reverse=True)[1:6] # Se toman los 5 siguientes a la misma palabra
        # Agregamos el verbo con su respectiva lista con palabras similares
        similar_words.append((most_frequent_verb, top_similar))
    # de lo contrario...
    else:
        # agregamos la tupla de la palabra con la leyenda correspondiente: "No disponible en GloVe"
        similar_words.append((most_frequent_verb, "No disponible en GloVe"))

# Imprimimos los resultados
for i, (verb, similarities) in enumerate(similar_words):
    print(f"Libro {i + 1} - Verbo más frecuente: '{verb}'")
    print("Palabras más similares:")
    if isinstance(similarities, str):
        print(similarities)
    else:
        for word, score in similarities:
            print(f"  {word}: {score:.4f}")
    print()

Libro 1 - Verbo más frecuente: 'say'
Palabras más similares:
  believe: 0.8163
  know: 0.7665
  want: 0.7577
  think: 0.7299
  might: 0.7227

Libro 2 - Verbo más frecuente: 'throw'
Palabras más similares:
  ball: 0.5623
  catch: 0.5044
  let: 0.4890
  go: 0.4826
  get: 0.4729

Libro 3 - Verbo más frecuente: 'go'
Palabras más similares:
  come: 0.7988
  do: 0.7331
  take: 0.7206
  would: 0.6825
  think: 0.6638

Libro 4 - Verbo más frecuente: 'take'
Palabras más similares:
  give: 0.7691
  would: 0.7523
  make: 0.7344
  come: 0.7306
  go: 0.7206

Libro 5 - Verbo más frecuente: 'take'
Palabras más similares:
  give: 0.7691
  would: 0.7523
  come: 0.7306
  could: 0.7007
  turn: 0.6617



## Similitud de documentos con “embedding”
Se seguirá la indicación y se descargará el modelo `bert-base-uncased` y se buscará encontrar la similitud entre las frases principales del primer libro con el resto.

In [17]:
# Cargamos los métodos y modelos necesarios parausar BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [18]:
def encode_text(text: str):
    # Tokenización y preparación para BERT
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Guardamos los resultados del modelo
    with torch.no_grad():
        outputs = model(**inputs)
    # Representación de embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    # Regresamos la interpretación
    return embeddings

In [19]:
# Definimos una función para obtener la similitud
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)

In [20]:
# Codificamos las frases
top_sentences_embeddings = [encode_text(sentence) for sentence in top_sentences]
# Almacenamos los puntajes en una lista y calculamos las similitudes
bert_grades: list = [calculate_similarity(top_sentences_embeddings[0], emb) for emb in top_sentences_embeddings]

# Imprimimos los resultados
for grade, sentence in zip(bert_grades, top_sentences):
    print(f"La similitud entre \"{top_sentences[0]}\" y \"{sentence}\" es: {grade:.4f}")

La similitud entre "classify six hundred different geological specimens" y "classify six hundred different geological specimens" es: 1.0000
La similitud entre "classify six hundred different geological specimens" y "several thousand people crushed" es: 0.5981
La similitud entre "classify six hundred different geological specimens" y "immense kraken whose tentacles could entangle" es: 0.6351
La similitud entre "classify six hundred different geological specimens" y "two officers listened gravely enough" es: 0.5409
La similitud entre "classify six hundred different geological specimens" y "car soon came back empty" es: 0.5192
